In [1]:
import requests
import time
import pandas as pd
pd.set_option("display.max.columns", None)

## Congress
headers = {'X-API-Key': 'Li9aCR2BjLZviTE2vhRXQ9QDqq1XU8NxXUXkQDbA',}
pp_base = 'https://api.propublica.org/congress/v1/'

# GET https://api.propublica.org/congress/v1/{congress}/bills/{bill-id}.json

CONGRESS = 114

bill_id = 'sres642'

data = requests.get(f'{pp_base}{CONGRESS}/bills/{bill_id}.json', headers = headers).json()
data

{'status': 'OK',
 'copyright': 'Copyright (c) 2023 Pro Publica Inc. All Rights Reserved.',
 'results': [{'bill_id': 'sres642-114',
   'bill_slug': 'sres642',
   'congress': '114',
   'bill': 'S.RES.642',
   'bill_type': 'sres',
   'number': 'S.RES.642',
   'bill_uri': 'https://api.propublica.org/congress/v1/114/bills/sres642.json',
   'title': 'A resolution authorizing taking pictures and filming in the Senate Chamber, the Senate Wing of the United States Capitol, and Senate Office Buildings for production of a film and a book on the history of the Senate.',
   'short_title': 'A resolution authorizing taking pictures and filming in the Senate Chamber, the Senate Wing of the United States Capitol, and Senate Office Buildings for production of a film and a book on the history of the Senate.',
   'sponsor_title': 'Sen.',
   'sponsor': 'Mitch McConnell',
   'sponsor_id': 'M000355',
   'sponsor_uri': 'https://api.propublica.org/congress/v1/members/M000355.json',
   'sponsor_party': 'R',
   

In [2]:
data['results'][0]['summary']

'(This measure has not been amended since it was introduced. The summary of that version is repeated here.) Suspends specified provisions of Rule IV of the Rules for the Regulation of the Senate Wing of the U.S. Capitol (prohibiting the taking of pictures in the Senate Chamber) to permit the taking of pictures and filming while the Senate is in session or in recess beginning on the adoption of this resolution and ending on May 1, 2017. Requires the Majority and the Minority Leaders of the Senate, and the Chairman and Ranking Member of the Senate Committee on Rules and Administration to enter into a memorandum of understanding with an individual or entity seeking to take such pictures and conduct such filming to produce a film documentary and a book on the history of the Senate to formalize an agreement on: (1) locations and times for taking pictures and conducting filming, and (2) the use of the pictures taken and film made.'

In [3]:
# Read the CSV file into a pandas dataframe
dfd = pd.read_csv('bills_database/bills_93-114_dem.csv')
dfr = pd.read_csv('bills_database/bills_93-114_rep.csv')
minor = pd.read_csv('bills_database/minor.csv', header=None)
major = pd.read_csv('bills_database/major.csv')

# Create code dictionaries
minor[['code', 'description']] = minor[0].str.split(":", n=1, expand=True)
minor = minor.drop(0, axis=1)
minor = minor.set_index('code').to_dict('index')
minor = {i:val['description'] for i, val in minor.items()}
major = major.set_index('code').to_dict('index')
major = {i:val['description'] for i, val in major.items()}

# Display the dataframe
df = pd.concat([dfd, dfr])

# Propublica only has info from congress 113
df = df.loc[df['Cong'] >= 113]


In [4]:
df['idx'] = df['BillType'] + df['BillNum'].astype(str)
dict_bills = df.to_dict('records')

In [5]:
dict_bills[0]

{'BillID': '113-HCONRES-2',
 'BillNum': 2,
 'BillType': 'hconres',
 'Chamber': 0,
 'Cong': 113,
 'Cosponsr': 0.0,
 'IntrDate': '2013-01-03',
 'Mult': 0.0,
 'MultNo': 1.0,
 'PLaw': 0,
 'Private': 0,
 'Title': 'Expressing the sense of Congress that a commemorative postage stamp should be issued in honor of George Thomas """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""Mickey"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""" Leland.',
 'Veto': 0,
 'Class': nan,
 'ComC': 0.0,
 'ComR': 0.0,
 'CumHServ': 7304.0,
 'CumSServ': 0.0,
 'Delegate': 0,
 'District': 18.0,
 'DW1': '-0,468',
 'Fr

In [10]:
def mergedicts(dict1, dict2):
    '''
    Esta funcion ayuda a combinar diccionarios
    '''
    for k in set(dict1.keys()).union(dict2.keys()):
        if k in dict1 and k in dict2:
            if isinstance(dict1[k], dict) and isinstance(dict2[k], dict):
                yield (k, dict(mergedicts(dict1[k], dict2[k])))
            else:
                yield (k, dict2[k])
        elif k in dict1:
            yield (k, dict1[k])
        else:
            yield (k, dict2[k])

final_dta = []

def populate():
    for i in dict_bills:
        congress = i['Cong']
        bill_id = i['idx']
        print('congress:',congress,'// id:', bill_id)
        propublica_data = requests.get(f'{pp_base}{congress}/bills/{bill_id}.json', headers = headers).json()
        if propublica_data['status'] == 'ERROR':
            print(0)
            continue
        print('1')
        data = propublica_data['results'][0]
        if data['summary'] == '':
            continue
        print(data)
        data['Major_desc'] = major.get(i['Major'], None)
        data['Minor_desc'] = major.get(i['Minor'], None)
        final_dta.append(mergedicts(i,data))
        
populate()

congress: 113 // id: hconres2
1
{'bill_id': 'hconres2-113', 'bill_slug': 'hconres2', 'congress': '113', 'bill': 'H.CON.RES.2', 'bill_type': 'hconres', 'number': 'H.CON.RES.2', 'bill_uri': 'https://api.propublica.org/congress/v1/113/bills/hconres2.json', 'title': 'Expressing the sense of Congress that a commemorative postage stamp should be issued in honor of George Thomas "Mickey" Leland.', 'short_title': 'Expressing the sense of Congress that a commemorative postage stamp should be issued in honor of George Thomas "Mickey" Leland.', 'sponsor_title': 'Rep.', 'sponsor': 'Sheila Jackson Lee', 'sponsor_id': 'J000032', 'sponsor_uri': 'https://api.propublica.org/congress/v1/members/J000032.json', 'sponsor_party': 'D', 'sponsor_state': 'TX', 'gpo_pdf_uri': None, 'congressdotgov_url': 'https://www.congress.gov/bill/113th-congress/house-concurrent-resolution/2', 'govtrack_url': 'https://www.govtrack.us/congress/bills/113/hconres2', 'introduced_date': '2013-01-03', 'active': False, 'last_vote':

KeyError: 'status'

In [17]:
dict(final_dta[1])

{'latest_major_action_date': '2013-01-03',
 'PassH': 0,
 'BillID': '113-HCONRES-5',
 'Major': nan,
 'committee_codes': ['HSHA'],
 'bill_slug': 'hconres5',
 'vetoed': None,
 'summary_short': "Authorizes the use of Emancipation Hall in the Capitol Visitor Center on March 21, 2013, for an event to celebrate the Mississippi River and its status as a vital resource of the United States and an indelible component of the nation's environment, economy, cultural heritage, and history. ",
 'enacted': None,
 'MultNo': 1.0,
 'Major_desc': None,
 'sponsor': 'Tim Walz',
 'bill_type': 'hconres',
 'bill_uri': 'https://api.propublica.org/congress/v1/113/bills/hconres5.json',
 'sponsor_title': 'Rep.',
 'NameLast': 'Walz',
 'votes': [],
 'SubChRef': 0.0,
 'Cong': 113,
 'idx': 'hconres5',
 'Minor_desc': None,
 'Chamber': 0,
 'ComC': 0.0,
 'Minor': nan,
 'active': False,
 'senate_passage_vote': None,
 'cbo_estimate_url': None,
 'PLawNum': nan,
 'house_passage': None,
 'Gender': 0,
 'ChRef': 0,
 'FrstConH':